# 1. Establish communication with the card

## 1.1. Find the smartcard reader

In [1]:
from applets.helpers import get_connection, AppletBase

In [2]:
connection = get_connection()
if connection is None:
    print("Failed to open a connection. No cardreaders?")

## 1.2. Power on the card and get `ATR`
`ATR` - Answer To Reset, bytes retured by the card when it is powered on.<br>
It helps to determine communication protocol and frequency we should choose.

In [3]:
atr=bytes(connection.getATR())
print(atr.hex())

3bdc18ff8191fe1fc38073c821136605036351000250


## 1.3. Select the applet by AID
JavaCard can have multiple applets installed. They are like very isolated "apps".<br>
They can't talk to each other and only one can be active at any time.

In [4]:
from applets.teapot import Teapot

class MemoryCard(Teapot):
    def __init__(self, connection=None):
        super().__init__(connection)
        self.AID = "B00B5111CB01"

    def get_random(self):
        return self.request("B0B10000")

In [5]:
app = MemoryCard(connection)
app.select()

b''

In [6]:
app.get_random().hex()

'39bbacb11b2f1485b155be66f2272780c667353fc98bdb6ae678ec205801b7fc'

# 2. Establish secure communication

At the moment we use a homebrew version, I need to understand and implement Noise in the near future.

[Noise protocol](https://noiseprotocol.org/noise.html) should be with the following primitives:
- secp256k1 as signature scheme
- AES CBC with random IV as a symmetric cipher
- SHA256 as a hash function, HMAC_SHA256 as authentication code

AES CBC because we have hardware implementation on the card, so we can use the Key class that is more secure than byte array. HMAC sucks here though...

Generally a shared secret is split to 2 different secrets for encryption and decryption. This allows us to use AES in counter mode where the counter is the same for both host and the card - they use different keys so it's ok.

Description of the current protocol:
1. Get the static public key from the card. If it is already known - proceed with step 2
2. Generate a random key pair and send corresponding pubkey in uncompressed form to the card: `<data:pub_uncompressed>`
3. Card will return its fresh session public key, hmac of this public key using shared secret and signature: `<data:session_pubkey_card><hmac(dec,data)><sig(static_pubkey_card,data|hmac)>`
4. Verify the signature against card's pubkey
5. Calculate a pair of shared secrets from `enc=SHA256('host'|ECDH(d,session_pubkey_card))`, `dec=SHA256('card'|ECDH(d,session_pubkey_card)`
6. Verify that `hmac==HMAC_SHA256(dec,session_pubkey_card)`
7. Set counter to zero `iv=0`, use `dec` for decryption, `enc` for enctyption. Increase the counter on every request, reestablish communication if the counter reaches max value (16 bytes, shouldn't happen in most cases)
8. All secure messages now should be in the form `<ct:aes(enc,iv,command|data)><hmac_sha256(enc,ct)>`, `iv` is not transmitted but increased for every new command. Returned data will be of the same form but using the `dec` key. `iv` will be the same.

Maximum size of the APDU is 256 bytes. Maximum size of the mnemonic is `9*24-1=215` bytes plus length and command and rounding to `16` bytes, plus `32`-byte hmac - we are at `256` bytes... We can send at max `255` bytes.

So to store and retrieve large mnemonic we need to split it to two pieces, and provide a function to set part of the data array. Therefore for secret storage we have to make an API: 
- `set_data<data>` / `get_data` - for short mnemonics
- `set_data_length<len>` - should be called first, erases existing data, 
- `set_data_part <start><data>` - stores part of the data with an offset
- `get_data_length`
- `get_data_part <start><len>` - gets part of the data from `start` with `len`


## 2.1. Get public key from the card

In [7]:
# embit library: https://github.com/diybitcoinhardware/embit
# works with python3 and micropython
from embit.ec import PrivateKey, PublicKey, secp256k1, Signature

In [8]:
# get pubkey - private key is generated when applet is installed
sec = app.request("B0B2000000")
card_pubkey = PublicKey.parse(sec)
card_pubkey

PublicKey(04d380946cfa081f4e3fc48f9adc540c9959c016a291145b537747cceb33a497e593803a1399bcd3a0ff62a78fbd378294d1c7c167a1573904bcd40c018dc1124a)

## 2.2. Select a random secret on the host

In [33]:
import os
import hashlib
secret = os.urandom(32)

In [34]:
host_prv = PrivateKey(secret)
host_pub = host_prv.get_public_key()
host_pub.compressed = False
data = bytes([65])+host_pub.sec()
data.hex()

'4104a47517d9832b074b742f3467e2b8f6abb2fe90280f3560398d1bcbec5a7e53342c81f59f37c45b7cc0f4d3bc5099f94966a8e43c877b509f6cd4e3408fe12219'

In [35]:
# ugly copy...
pub = PublicKey.parse(card_pubkey.sec())
secp256k1.ec_pubkey_tweak_mul(pub._point, secret)
shared_secret = pub.sec()[1:33]
shared_hash = hashlib.sha256(shared_secret).digest()
print("Shared secret:", shared_secret.hex())
print("Sha256 of the secret:", shared_hash.hex())

Shared secret: 2c9e072a65b417d4c881f3a00d2ba05a623f1847a5ba7b988a6993d9fd61734d
Sha256 of the secret: cfb12637e9c029a735b299f21e7797e42cdd3f6d2108ad7fa521a72ced6e6abf


In [36]:
res = app.request("B0B30000"+data.hex())
print(res.hex())

cfb12637e9c029a735b299f21e7797e42cdd3f6d2108ad7fa521a72ced6e6abf5d690ac3fdef606a3231834e4126c2a6ee076a40a617fe4af7343aeeda70825d30460221008fd85d3fb8326cc0a2f267bcc17dd0d5e45d2138d57f4e19553e89b0155d8482022100909d32329d1e376f9746e6110502c2ad26d5ee3d0ba6683acffca62f8b0ef277


In [43]:
import hmac
recv_hmac = res[32:64]
h = hmac.new(shared_secret, digestmod='sha256')
h.update(res[:32])
expected_hmac = h.digest()
if expected_hmac != recv_hmac:
    raise RuntimeError("Wrong HMAC. Got %s, expected %s"%(recv_hmac.hex(),expected_hmac.hex()))
sig = Signature.parse(res[64:])
# card doesn't follow low s rule, so we need to normalize
sig._sig = secp256k1.ecdsa_signature_normalize(sig._sig)
if not card_pubkey.verify(sig, hashlib.sha256(res[:64]).digest()):
    raise RuntimeError("Signature is invalid")
if res[:32] == shared_hash:
    print("Yey! We have a shared secret now!")
else:
    print("Meh... Something didn't work? Card returned %s, we have %s" % (res.hex(), shared_hash.hex()))

Yey! We have a shared secret now!


In [45]:
# alternative ecdh:
res = app.request("B0B40000"+data.hex())
print(res.hex())

044542afd9776ee4fa5e4f81bbaebcdb0a7c23d296108734fdbb612c637ea7633481553071e31fed576ff181b0e2fbc9fba45161da12fbf9b58a1fe99457a0a9f264d58ac78b4a4d7eb51c8a26f190f586edc407f9f727b51de34cda830ac2aa10304402204cf24733b7cf4deeb27c097c66ddfbfc4f460e6b9cc8082389276ebfbac7a20b0220591d21400b359d693d8b8cc0102edaf5274fa53b55f0c97c4d61cd8dd37b2adf


In [46]:
pub = PublicKey.parse(res[:65])
secp256k1.ec_pubkey_tweak_mul(pub._point, secret)
shared_secret = pub.sec()[1:33]
shared_hash = hashlib.sha256(shared_secret).digest()
print("Shared secret:", shared_secret.hex())
print("Sha256 of the secret:", shared_hash.hex())

Shared secret: 1383a679d9185c8f419586f1a1d572010c28d1c881248a03bb3d2d0386281480
Sha256 of the secret: 1e9c5e870cac618b334124e0affeb065240e24b249f2ae5c7f0749284845fd79


In [48]:
recv_hmac = res[65:97]
h = hmac.new(shared_secret, digestmod='sha256')
h.update(res[:65])
expected_hmac = h.digest()
if expected_hmac != recv_hmac:
    raise RuntimeError("Wrong HMAC. Got %s, expected %s"%(recv_hmac.hex(),expected_hmac.hex()))
sig = Signature.parse(res[97:])
# card doesn't follow low s rule, so we need to normalize
sig._sig = secp256k1.ecdsa_signature_normalize(sig._sig)
if not card_pubkey.verify(sig, hashlib.sha256(res[:97]).digest()):
    raise RuntimeError("Signature is invalid")
print("Yey! We have a shared secret now!")

Yey! We have a shared secret now!


In [44]:
# get random data
rand = app.request("B0B2000000")
rand.hex()

'04d380946cfa081f4e3fc48f9adc540c9959c016a291145b537747cceb33a497e593803a1399bcd3a0ff62a78fbd378294d1c7c167a1573904bcd40c018dc1124a'

In [ ]:
# static data for any other APDU
data = app.request("B0A1000000")
data

In [ ]:
(bytes([len(data)])+data).hex()

In [ ]:
# static data for any other APDU
c = app.request("B0A40000"+(bytes([len(data)])+data).hex())
c

In [ ]:
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
from cryptography.hazmat.backends import default_backend
import hmac

def decrypt(c,secret):
    recv_hmac = c[-32:]
    iv = c[:16]
    ct = c[16:-32]
    h = hmac.new(shared_secret, digestmod='sha256')
    h.update(iv)
    h.update(ct)
    expected_hmac = h.digest()
    if expected_hmac != recv_hmac:
        raise RuntimeError("Wrong HMAC. Got %s, expected %s"%(recv_hmac.hex(),expected_hmac.hex()))
    backend = default_backend()
    cipher = Cipher(algorithms.AES(secret), modes.CBC(iv), backend=backend)
    decryptor = cipher.decryptor()
    # remove \x80... padding
    plain = decryptor.update(ct)+decryptor.finalize()
    return b"".join(plain.split(b"\x80")[:-1])

decrypt(c, shared_secret)

In [49]:
connection.disconnect()